#### Load the package, dataset and set the parameter

In [12]:
import pandas as pd
import numpy as np

In [13]:
# 读取数据
df = pd.read_csv("Sample Data.csv")

# 参数设置
max_weight_FCL40 = 30480  # FCL40的最大承重，单位：kg
max_volume_FCL40 = 67     # FCL40的空间体积，单位：CBM
cost_FCL40 = 5600         # FCL40的价格

max_weight_FCL20 = 14969  # FCL20的最大承重，单位：kg
max_volume_FCL20 = 33     # FCL20的空间体积，单位：CBM
cost_FCL20 = 3500         # FCL20的价格

cost_LCL = 778            # LCL的价格，单位：每CBM

# 随机变量的设置
np.random.seed(0)  # 设置随机种子
transport_time_FCL = np.random.randint(19, 23)  # FCL运输时间变量
transport_time_LCL = np.random.randint(25, 31)  # LCL运输时间变量
weather_risk = np.random.choice([0, 1, 2, 3], p=[0.97, 0.01, 0.01, 0.01])  # 天气风险变量


#### Initial Check on items

In [14]:
# 计算所有货物的总重量和总体积
total_weight = df['Weight(kg)'].sum()
total_volume = df['Volume(CBM)'].sum()

# 判断是否需要FCL40，如果不需要，则考虑FCL20或LCL
needs_FCL40 = total_weight > max_weight_FCL20 or total_volume > max_volume_FCL20
needs_FCL20_or_LCL = not needs_FCL40

# 如果需要FCL40，我们将在后面的步骤中分配货物到FCL40
# 如果不需要FCL40，我们将在后续步骤中决定是使用FCL20还是LCL
print(f"需要使用FCL40: {needs_FCL40}")
print(f"可能使用FCL20或LCL: {needs_FCL20_or_LCL}")

需要使用FCL40: True
可能使用FCL20或LCL: False


#### Planning to assign items

In [15]:
def fill_container(df, max_weight, max_volume):
    """Fill the container by selecting a set of items that maximizes the usage of space."""
    # Create a DataFrame to keep track of which items are in the container
    container_df = pd.DataFrame(columns=df.columns)
    
    # Sort the items by volume to weight ratio in descending order
    df['volume_to_weight_ratio'] = df['Volume(CBM)'] / df['Weight(kg)']
    sorted_items = df.sort_values(by='volume_to_weight_ratio', ascending=False)
    
    remaining_weight = max_weight
    remaining_volume = max_volume
    
    # Attempt to add items to the container until there's no more room
    for _, item in sorted_items.iterrows():
        if item['Weight(kg)'] <= remaining_weight and item['Volume(CBM)'] <= remaining_volume:
            container_df = container_df.append(item)
            remaining_weight -= item['Weight(kg)']
            remaining_volume -= item['Volume(CBM)']
    
    return container_df, remaining_weight, remaining_volume

# Attempt to fill an FCL40 container
items_to_load = df.copy()  # Assuming df is your DataFrame containing items
filled_container, weight_left, volume_left = fill_container(items_to_load, max_weight_FCL40, max_volume_FCL40)

print("Items loaded into the FCL40 container:")
print(filled_container)
print(f"Weight left in container: {round(weight_left,2)} kg")
print(f"Volume left in container: {round(volume_left,2)} CBM")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local

Items loaded into the FCL40 container:
      ID Volume(CBM) Weight(kg) Time Requirement(Day) Risk Maximum  \
16  17.0         4.8      978.0                  24.0          2.0   
14  15.0         4.8     1007.0                  25.0          2.0   
15  16.0         2.9      620.0                  27.0          3.0   
17  18.0         3.8      814.0                  26.0          2.0   
8    9.0         4.6     1020.0                  20.0          2.0   
0    1.0         3.9      895.0                  18.0          2.0   
10  11.0         3.9      904.0                  19.0          2.0   
1    2.0         8.5     2065.0                  20.0          2.0   
6    7.0         5.8     1457.0                  22.0          2.0   
19  20.0         6.7     1777.0                  25.0          2.0   
2    3.0         6.8     1820.0                  29.0          3.0   
9   10.0         2.8      755.0                  28.0          1.0   
11  12.0         7.7     2086.0                  24

C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)


In [5]:
def allocate_items_to_containers(df, max_weight_FCL40, max_volume_FCL40, max_weight_FCL20, max_volume_FCL20, cost_FCL40, cost_FCL20, cost_LCL):
    # 初始化决策矩阵和集装箱使用情况
    decision_matrix = pd.DataFrame(0, index=df.index, columns=['FCL40', 'FCL20', 'LCL'])
    containers_usage = {'FCL40': {'count': 0, 'weight': [], 'volume': []},
                        'FCL20': {'count': 0, 'weight': [], 'volume': []},
                        'LCL': {'volume': 0}}

    # 排序货物以优化装载
    df['volume_to_weight_ratio'] = df['Volume(CBM)'] / df['Weight(kg)']
    remaining_items = df.sort_values(by='volume_to_weight_ratio', ascending=False)

    # 一直分配直到所有货物都被分配
    while not remaining_items.empty:
        # 尝试填满FCL40
        filled_FCL40, rem_weight_FCL40, rem_volume_FCL40 = fill_container(remaining_items, max_weight_FCL40, max_volume_FCL40)
        if not filled_FCL40.empty:
            decision_matrix.loc[filled_FCL40.index, 'FCL40'] = 1
            containers_usage['FCL40']['count'] += 1
            containers_usage['FCL40']['weight'].append(max_weight_FCL40 - rem_weight_FCL40)
            containers_usage['FCL40']['volume'].append(max_volume_FCL40 - rem_volume_FCL40)
            remaining_items = remaining_items.drop(filled_FCL40.index)

        # 尝试填满FCL20
        filled_FCL20, rem_weight_FCL20, rem_volume_FCL20 = fill_container(remaining_items, max_weight_FCL20, max_volume_FCL20)
        if not filled_FCL20.empty:
            decision_matrix.loc[filled_FCL20.index, 'FCL20'] = 1
            containers_usage['FCL20']['count'] += 1
            containers_usage['FCL20']['weight'].append(max_weight_FCL20 - rem_weight_FCL20)
            containers_usage['FCL20']['volume'].append(max_volume_FCL20 - rem_volume_FCL20)
            remaining_items = remaining_items.drop(filled_FCL20.index)

        # 如果无法再装入FCL20或FCL40，分配至LCL
        if not remaining_items.empty:
            decision_matrix.loc[remaining_items.index, 'LCL'] = 1
            containers_usage['LCL']['volume'] += remaining_items['Volume(CBM)'].sum()
            break

    # 计算总成本
    total_cost = (containers_usage['FCL40']['count'] * cost_FCL40 +
                  containers_usage['FCL20']['count'] * cost_FCL20 +
                  containers_usage['LCL']['volume'] * cost_LCL)

    return decision_matrix, containers_usage, total_cost

# 分配货物至集装箱并计算总成本
decision_matrix, containers_usage, total_cost = allocate_items_to_containers(
    df, max_weight_FCL40, max_volume_FCL40, max_weight_FCL20, max_volume_FCL20, cost_FCL40, cost_FCL20, cost_LCL
)

# 打印决策矩阵和总成本
print(decision_matrix)
print(f"集装箱使用情况: {containers_usage}")
print(f"总成本: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local

    FCL40  FCL20  LCL
0       1      0    0
1       1      0    0
2       1      0    0
3       0      1    0
4       0      1    0
5       0      0    1
6       1      0    0
7       0      1    0
8       1      0    0
9       1      0    0
10      1      0    0
11      1      0    0
12      0      0    1
13      0      0    1
14      1      0    0
15      1      0    0
16      1      0    0
17      1      0    0
18      0      1    0
19      1      0    0
集装箱使用情况: {'FCL40': {'count': 1, 'weight': [16198.0], 'volume': [66.99999999999999]}, 'FCL20': {'count': 1, 'weight': [8746.0], 'volume': [30.8]}, 'LCL': {'volume': 25.2}}
总成本: 28705.6


C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_20632\982020246.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(item)
C:\Users\37605\AppData\Local

In [16]:
def fill_container(df, max_weight, max_volume):
    """尝试填满一个集装箱，直到不能再装更多的货物为止。"""
    # 初始化集装箱内的货物信息
    container_items = pd.DataFrame(columns=df.columns)
    # 当前集装箱的剩余承重和容积
    remaining_weight = max_weight
    remaining_volume = max_volume
    
    # 对剩余货物按体积和重量的比率进行排序
    df['ratio'] = df['Volume(CBM)'] / df['Weight(kg)']
    sorted_items = df.sort_values('ratio', ascending=False)
    
    # 尝试将每件货物装入集装箱
    for _, item in sorted_items.iterrows():
        if item['Weight(kg)'] <= remaining_weight and item['Volume(CBM)'] <= remaining_volume:
            container_items = container_items.append(item)
            remaining_weight -= item['Weight(kg)']
            remaining_volume -= item['Volume(CBM)']

    # 返回已装入集装箱的货物和未装入的货物
    remaining_items = df.drop(container_items.index)
    return container_items, remaining_items, remaining_weight, remaining_volume

# 主要的货物分配函数
def allocate_items(df, max_weight_FCL20, max_volume_FCL20, max_weight_FCL40, max_volume_FCL40, cost_FCL20, cost_FCL40, cost_LCL):
    # 初始化集装箱使用情况
    containers = {'FCL40': [], 'FCL20': [], 'LCL': 0}
    costs = {'FCL40': 0, 'FCL20': 0, 'LCL': 0}

    while not df.empty:
        if df['Volume(CBM)'].sum() <= max_volume_FCL20 and df['Weight(kg)'].sum() <= max_weight_FCL20:
            # 如果所有货物都可以装入FCL20
            items, df, weight_left, volume_left = fill_container(df, max_weight_FCL20, max_volume_FCL20)
            containers['FCL20'].append((weight_left, volume_left))
            costs['FCL20'] += cost_FCL20
        else:
            # 如果需要装入FCL40
            items, df, weight_left, volume_left = fill_container(df, max_weight_FCL40, max_volume_FCL40)
            containers['FCL40'].append((weight_left, volume_left))
            costs['FCL40'] += cost_FCL40

        # 如果剩余货物体积小且FCL20的成本较高，考虑LCL
        if not df.empty and (df['Volume(CBM)'].sum() * cost_LCL < cost_FCL20):
            containers['LCL'] += df['Volume(CBM)'].sum()
            costs['LCL'] += df['Volume(CBM)'].sum() * cost_LCL
            break

    total_cost = costs['FCL40'] + costs['FCL20'] + costs['LCL']
    return containers, total_cost

# 执行货物分配
containers, total_cost = allocate_items(
    df, max_weight_FCL20, max_volume_FCL20, max_weight_FCL40, max_volume_FCL40, cost_FCL20, cost_FCL40, cost_LCL
)

print("集装箱使用情况:")
for container_type, usage in containers.items():
    print(f"{container_type}: {usage}")

print(f"总成本: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)


集装箱使用情况:
FCL40: [(14282.0, 1.0658141036401503e-14), (13617.0, 11.000000000000002)]
FCL20: []
LCL: 0
总成本: 11200


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)


In [19]:
import pandas as pd
import numpy as np

# 假设df是包含所有货物信息的DataFrame
df = pd.read_csv('Sample Data.csv')

# 参数设置
max_weight_FCL40 = 30480  # FCL40的最大承重，单位：kg
max_volume_FCL40 = 67     # FCL40的空间体积，单位：CBM
cost_FCL40 = 5600         # FCL40的价格

max_weight_FCL20 = 14969  # FCL20的最大承重，单位：kg
max_volume_FCL20 = 33     # FCL20的空间体积，单位：CBM
cost_FCL20 = 3500         # FCL20的价格

cost_LCL = 778            # LCL的价格，单位：每CBM

# 随机变量的设置
np.random.seed(0)  # 设置随机种子
transport_time_FCL = np.random.randint(19, 23)  # FCL运输时间变量
transport_time_LCL = np.random.randint(25, 31)  # LCL运输时间变量
weather_risk = np.random.choice([0, 1, 2, 3], p=[0.97, 0.01, 0.01, 0.01])  # 天气风险变量

# 使用一个贪婪算法，该算法按体积与重量比例从大到小排序货物，并尝试填充集装箱
def greedy_fill_container(df, max_weight, max_volume):
    # 按照体积与重量的比率对货物进行排序，比率大的优先装载
    df['ratio'] = df['Volume(CBM)'] / df['Weight(kg)']
    items_sorted = df.sort_values('ratio', ascending=False)
    
    # 逐个尝试装载货物，直到不能装载更多货物为止
    loaded_items = []
    total_volume = 0
    total_weight = 0

    for _, item in items_sorted.iterrows():
        if total_volume + item['Volume(CBM)'] <= max_volume and total_weight + item['Weight(kg)'] <= max_weight:
            loaded_items.append(item['ID'])
            total_volume += item['Volume(CBM)']
            total_weight += item['Weight(kg)']

    # 生成装载和未装载货物的DataFrame
    loaded_df = df[df['ID'].isin(loaded_items)]
    remaining_df = df[~df['ID'].isin(loaded_items)]

    return loaded_df, remaining_df

# 循环分配货物到集装箱
def allocate_to_containers(df, max_weight_FCL20, max_volume_FCL20, max_weight_FCL40, max_volume_FCL40, cost_FCL20, cost_FCL40, cost_LCL):
    loaded_info = {'FCL40': [], 'FCL20': [], 'LCL': []}
    while not df.empty:
        # 首先尝试将货物装入FCL40
        loaded_df, df = greedy_fill_container(df, max_weight_FCL40, max_volume_FCL40)
        if not loaded_df.empty:
            loaded_info['FCL40'].append(loaded_df)
        else:
            # 如果FCL40装不下，尝试装入FCL20
            loaded_df, df = greedy_fill_container(df, max_weight_FCL20, max_volume_FCL20)
            if not loaded_df.empty:
                loaded_info['FCL20'].append(loaded_df)
            else:
                # 如果FCL20也装不下，剩余货物装入LCL
                loaded_info['LCL'].append(df)
                break

    # 计算总成本
    total_cost = (len(loaded_info['FCL40']) * cost_FCL40 +
                  len(loaded_info['FCL20']) * cost_FCL20 +
                  sum([items['Volume(CBM)'].sum() for items in loaded_info['LCL']]) * cost_LCL)

    return loaded_info, total_cost

In [20]:
# 执行分配函数
loaded_info, total_cost = allocate_to_containers(
    df, 
    max_weight_FCL20, max_volume_FCL20, 
    max_weight_FCL40, max_volume_FCL40, 
    cost_FCL20, cost_FCL40, cost_LCL
)

# 打印分配后的装载信息和总成本
print("装载信息：")
for container_type, containers in loaded_info.items():
    for container in containers:
        print(f"{container_type} 装载的货物ID: {container['ID'].tolist()}")
        print(f"{container_type} 总重量: {container['Weight(kg)'].sum()} kg")
        print(f"{container_type} 总体积: {container['Volume(CBM)'].sum()} CBM\n")

print(f"总成本: {total_cost}")


装载信息：
FCL40 装载的货物ID: [1, 2, 9, 10, 11, 15, 16, 17, 18, 21, 22, 23, 24, 25]
FCL40 总重量: 14353 kg
FCL40 总体积: 64.4 CBM

FCL40 装载的货物ID: [3, 4, 5, 7, 8, 12, 14, 19, 20]
FCL40 总重量: 17751 kg
FCL40 总体积: 63.400000000000006 CBM

FCL40 装载的货物ID: [6, 13]
FCL40 总重量: 6252 kg
FCL40 总体积: 19.6 CBM

总成本: 16800


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\1206771358.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratio'] = df['Volume(CBM)'] / df['Weight(kg)']


In [21]:
import pandas as pd
import numpy as np

# 假设df是包含所有货物信息的DataFrame
df = pd.read_csv('Sample Data.csv')

# 参数设置
max_weight_FCL40 = 30480  # FCL40的最大承重，单位：kg
max_volume_FCL40 = 67     # FCL40的空间体积，单位：CBM
cost_FCL40 = 5600         # FCL40的价格

max_weight_FCL20 = 14969  # FCL20的最大承重，单位：kg
max_volume_FCL20 = 33     # FCL20的空间体积，单位：CBM
cost_FCL20 = 3500         # FCL20的价格

cost_LCL = 778            # LCL的价格，单位：每CBM

def allocate_items(df, max_weight_FCL20, max_volume_FCL20, max_weight_FCL40, max_volume_FCL40, cost_FCL20, cost_FCL40, cost_LCL):
    # 初始化决策矩阵和集装箱使用情况
    decision_matrix = pd.DataFrame(0, index=df.index, columns=['FCL40', 'FCL20', 'LCL'])
    container_usage = {'FCL40': {'count': 0, 'weight': [], 'volume': []},
                       'FCL20': {'count': 0, 'weight': [], 'volume': []},
                       'LCL': {'volume': 0}}
    
    while not df.empty:
        total_volume = df['Volume(CBM)'].sum()
        total_weight = df['Weight(kg)'].sum()
        
        # 判断是使用FCL40还是FCL20
        if total_volume > max_volume_FCL20 or total_weight > max_weight_FCL20:
            # 如果超出FCL20的限制，优先使用FCL40
            container_df, df = fill_container(df, max_weight_FCL40, max_volume_FCL40)
            decision_matrix.loc[container_df.index, 'FCL40'] = 1
            container_usage['FCL40']['count'] += 1
            container_usage['FCL40']['weight'].append(max_weight_FCL40 - container_df['Weight(kg)'].sum())
            container_usage['FCL40']['volume'].append(max_volume_FCL40 - container_df['Volume(CBM)'].sum())
        else:
            # 如果不超出FCL20的限制，比较FCL20和LCL的成本
            fcl20_cost = cost_FCL20
            lcl_cost = total_volume * cost_LCL
            
            if fcl20_cost <= lcl_cost:
                # 如果FCL20的成本更低，使用FCL20
                container_df, df = fill_container(df, max_weight_FCL20, max_volume_FCL20)
                decision_matrix.loc[container_df.index, 'FCL20'] = 1
                container_usage['FCL20']['count'] += 1
                container_usage['FCL20']['weight'].append(max_weight_FCL20 - container_df['Weight(kg)'].sum())
                container_usage['FCL20']['volume'].append(max_volume_FCL20 - container_df['Volume(CBM)'].sum())
            else:
                # 如果LCL的成本更低，将剩余所有货物分配到LCL
                decision_matrix.loc[df.index, 'LCL'] = 1
                container_usage['LCL']['volume'] += total_volume
                df = pd.DataFrame()  # 清空剩余货物
                
    total_cost = (
        container_usage['FCL40']['count'] * cost_FCL40 +
        container_usage['FCL20']['count'] * cost_FCL20 +
        container_usage['LCL']['volume'] * cost_LCL
    )
    
    return decision_matrix, total_cost

# 执行货物分配
decision_matrix, total_cost = allocate_items(
    df, 
    max_weight_FCL20, max_volume_FCL20, 
    max_weight_FCL40, max_volume_FCL40, 
    cost_FCL20, cost_FCL40, cost_LCL
)

# 打印分配决策和总成本
print("分配决策矩阵:")
print(decision_matrix)
print(f"总成本: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2566167876.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)


ValueError: too many values to unpack (expected 2)

In [22]:
def fill_container(df, max_weight, max_volume):
    """尝试填满一个集装箱，直到不能再装更多的货物为止。"""
    # 初始化集装箱内的货物信息
    container_df = pd.DataFrame()
    remaining_weight = max_weight
    remaining_volume = max_volume

    for index, row in df.iterrows():
        if row['Weight(kg)'] <= remaining_weight and row['Volume(CBM)'] <= remaining_volume:
            container_df = container_df.append(row)
            remaining_weight -= row['Weight(kg)']
            remaining_volume -= row['Volume(CBM)']

    # 返回已装入集装箱的货物，以及剩余的货物，剩余的重量和体积
    return container_df, df.drop(container_df.index), remaining_weight, remaining_volume

def allocate_items(df, max_weight_FCL20, max_volume_FCL20, max_weight_FCL40, max_volume_FCL40, cost_FCL20, cost_FCL40, cost_LCL):
    # 初始化决策矩阵和集装箱使用情况
    decision_matrix = pd.DataFrame(0, index=df.index, columns=['FCL40', 'FCL20', 'LCL'])
    container_usage = {'FCL40': {'count': 0, 'weight': [], 'volume': []},
                       'FCL20': {'count': 0, 'weight': [], 'volume': []},
                       'LCL': {'volume': 0}}

    while not df.empty:
        total_volume = df['Volume(CBM)'].sum()
        total_weight = df['Weight(kg)'].sum()

        # 如果超出FCL20的限制，优先使用FCL40
        if total_volume > max_volume_FCL20 or total_weight > max_weight_FCL20:
            # 如果超出FCL20的限制，优先使用FCL40
            container_df, df = fill_container(df, max_weight_FCL40, max_volume_FCL40)
            decision_matrix.loc[container_df.index, 'FCL40'] = 1
            container_usage['FCL40']['count'] += 1
            container_usage['FCL40']['weight'].append(max_weight_FCL40 - container_df['Weight(kg)'].sum())
            container_usage['FCL40']['volume'].append(max_volume_FCL40 - container_df['Volume(CBM)'].sum())

        # 如果不超出FCL20的限制，比较FCL20和LCL的成本
        else:
             # 如果不超出FCL20的限制，比较FCL20和LCL的成本
            fcl20_cost = cost_FCL20
            lcl_cost = total_volume * cost_LCL

            if fcl20_cost <= lcl_cost:
                # 如果FCL20的成本更低，使用FCL20
                container_df, df = fill_container(df, max_weight_FCL20, max_volume_FCL20)
                decision_matrix.loc[container_df.index, 'FCL20'] = 1
                container_usage['FCL20']['count'] += 1
                container_usage['FCL20']['weight'].append(max_weight_FCL20 - container_df['Weight(kg)'].sum())
                container_usage['FCL20']['volume'].append(max_volume_FCL20 - container_df['Volume(CBM)'].sum())
            else:
                # 如果LCL的成本更低，将剩余所有货物分配到LCL
                decision_matrix.loc[df.index, 'LCL'] = 1
                container_usage['LCL']['volume'] += total_volume
                df = pd.DataFrame()  # 清空剩余货物
                
    total_cost = (
        container_usage['FCL40']['count'] * cost_FCL40 +
        container_usage['FCL20']['count'] * cost_FCL20 +
        container_usage['LCL']['volume'] * cost_LCL
    )
    

    return decision_matrix, total_cost

# 执行货物分配
decision_matrix, total_cost = allocate_items(
    df, 
    max_weight_FCL20, max_volume_FCL20, 
    max_weight_FCL40, max_volume_FCL40, 
    cost_FCL20, cost_FCL40, cost_LCL
)

# 打印分配决策和总成本
print("分配决策矩阵:")
print(decision_matrix)
print(f"总成本: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2855394468.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(row)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2855394468.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(row)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2855394468.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(row)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\2855394468.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_df = container_df.append(row)
C:\Users\37605\AppData\Local

ValueError: too many values to unpack (expected 2)

In [23]:
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_csv('Sample Data.csv')  # 请确保路径正确

# 货物分配到集装箱的函数
def fill_container(remaining_items, container_capacity):
    container_items = pd.DataFrame(columns=remaining_items.columns)
    remaining_capacity = container_capacity.copy()

    for index, item in remaining_items.iterrows():
        if item['Weight(kg)'] <= remaining_capacity['weight'] and item['Volume(CBM)'] <= remaining_capacity['volume']:
            container_items = container_items.append(item)
            remaining_capacity['weight'] -= item['Weight(kg)']
            remaining_capacity['volume'] -= item['Volume(CBM)']

    remaining_items = remaining_items.drop(container_items.index)
    return container_items, remaining_items, remaining_capacity

# 主分配函数
def allocate_items(items_df, container_specs, costs):
    allocated_containers = {
        'FCL40': [],
        'FCL20': [],
        'LCL': []
    }
    
    remaining_items = items_df.copy()
    while not remaining_items.empty:
        if remaining_items['Weight(kg)'].sum() <= container_specs['FCL20']['weight'] and remaining_items['Volume(CBM)'].sum() <= container_specs['FCL20']['volume']:
            # 可以使用FCL20
            filled_items, remaining_items, _ = fill_container(remaining_items, container_specs['FCL20'])
            allocated_containers['FCL20'].append(filled_items)
        elif remaining_items['Weight(kg)'].sum() > container_specs['FCL20']['weight'] or remaining_items['Volume(CBM)'].sum() > container_specs['FCL20']['volume']:
            # 需要使用FCL40
            filled_items, remaining_items, _ = fill_container(remaining_items, container_specs['FCL40'])
            allocated_containers['FCL40'].append(filled_items)
        else:
            # 剩余货物使用LCL
            allocated_containers['LCL'].append(remaining_items)
            break

    # 计算总成本
    total_cost = 0
    for container_type, container_list in allocated_containers.items():
        if container_type != 'LCL':
            total_cost += len(container_list) * costs[container_type]
        else:
            total_volume = sum([container['Volume(CBM)'].sum() for container in container_list])
            total_cost += total_volume * costs['LCL']

    return allocated_containers, total_cost

# 容器规格和成本
container_specs = {
    'FCL40': {'weight': max_weight_FCL40, 'volume': max_volume_FCL40},
    'FCL20': {'weight': max_weight_FCL20, 'volume': max_volume_FCL20}
}
costs = {
    'FCL40': cost_FCL40,
    'FCL20': cost_FCL20,
    'LCL': cost_LCL
}

# 运行分配
allocated_containers, total_cost = allocate_items(df, container_specs, costs)

# 输出结果
print("Allocated containers:")
for container_type, containers in allocated_containers.items():
    for container in containers:
        print(f"{container_type}: {container.index.tolist()}")
print(f"Total cost: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\U

Allocated containers:
FCL40: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
FCL40: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
FCL20: [21, 22, 23, 24]
Total cost: 14700


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\631712401.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)


In [24]:
import pandas as pd
import numpy as np

# 读取数据
df = pd.read_csv('Sample Data.csv')  # 请确保路径正确

# 容器规格和成本
max_weight_FCL40 = 30480  # FCL40的最大承重，单位：kg
max_volume_FCL40 = 67     # FCL40的空间体积，单位：CBM
cost_FCL40 = 5600         # FCL40的价格

max_weight_FCL20 = 14969  # FCL20的最大承重，单位：kg
max_volume_FCL20 = 33     # FCL20的空间体积，单位：CBM
cost_FCL20 = 3500         # FCL20的价格

cost_LCL = 778            # LCL的价格，单位：每CBM


In [25]:
def fill_container_greedy(items, max_weight, max_volume):
    # 将物品按体积与重量比进行排序
    items = items.assign(ratio=items['Volume(CBM)'] / items['Weight(kg)']).sort_values(by='ratio', ascending=False)
    container_items = pd.DataFrame(columns=items.columns)
    used_volume = 0
    used_weight = 0

    for _, item in items.iterrows():
        if used_volume + item['Volume(CBM)'] <= max_volume and used_weight + item['Weight(kg)'] <= max_weight:
            container_items = container_items.append(item)
            used_volume += item['Volume(CBM)']
            used_weight += item['Weight(kg)']

    # 返回集装箱内物品和剩余物品
    remaining_items = items.drop(container_items.index)
    return container_items, remaining_items


In [ ]:
def allocate_items(df, specs, costs):
    allocated = {'FCL40': [], 'FCL20': [], 'LCL': []}
    total_cost = 0

    while not df.empty:
        # 检查是否可以使用FCL20或LCL
        if df['Volume(CBM)'].sum() <= specs['FCL20']['volume'] and df['Weight(kg)'].sum() <= specs['FCL20']['weight']:
            filled, df = fill_container_greedy(df, specs['FCL20']['weight'], specs['FCL20']['volume'])
            allocated['FCL20'].append(filled)
            total_cost += costs['FCL20']
        else:
            filled, df = fill_container_greedy(df, specs['FCL40']['weight'], specs['FCL40']['volume'])
            if filled.empty:
                # 如果FCL40也无法装载，使用LCL
                allocated['LCL'].append(df)
                total_cost += df['Volume(CBM)'].sum() * costs['LCL']
                break
            else:
                allocated['FCL40'].append(filled)
                total_cost += costs['FCL40']

    return allocated, total_cost

# 定义集装箱规格和成本
specs = {
    'FCL40': {'weight': max_weight_FCL40, 'volume': max_volume_FCL40},
    'FCL20': {'weight': max_weight_FCL20, 'volume': max_volume_FCL20}
}
costs = {
   


In [26]:
import pandas as pd
import numpy as np

# 伪造数据加载
df = pd.read_csv('Sample Data.csv')  # 请确保路径正确

# 填充集装箱函数，使用贪婪算法按体积与重量比进行排序并填充
def fill_container_greedy(remaining_items, max_weight, max_volume):
    # 计算体积与重量比，并按比值降序排序
    remaining_items['ratio'] = remaining_items['Volume(CBM)'] / remaining_items['Weight(kg)']
    items_sorted = remaining_items.sort_values('ratio', ascending=False)

    container_items = pd.DataFrame()
    total_weight = total_volume = 0

    # 尝试将每件货物装入集装箱，直到集装箱满或没有货物可以装入为止
    for _, item in items_sorted.iterrows():
        if total_weight + item['Weight(kg)'] <= max_weight and total_volume + item['Volume(CBM)'] <= max_volume:
            container_items = container_items.append(item)
            total_weight += item['Weight(kg)']
            total_volume += item['Volume(CBM)']
    
    # 确定剩余的物品
    remaining_items = remaining_items.drop(container_items.index)
    return container_items, remaining_items

# 主分配函数，决定每个货物放入哪种集装箱
def allocate_items(df, specs, costs):
    allocation = {'FCL40': [], 'FCL20': [], 'LCL': []}
    remaining_items = df.copy()

    while not remaining_items.empty:
        if (remaining_items['Weight(kg)'].sum() <= specs['FCL20']['weight'] and
                remaining_items['Volume(CBM)'].sum() <= specs['FCL20']['volume']):
            # 能装入FCL20
            container_items, remaining_items = fill_container_greedy(remaining_items, specs['FCL20']['weight'], specs['FCL20']['volume'])
            allocation['FCL20'].append(container_items)
        elif (remaining_items['Weight(kg)'].sum() > specs['FCL20']['weight'] or
              remaining_items['Volume(CBM)'].sum() > specs['FCL20']['volume']):
            # 需要使用FCL40
            container_items, remaining_items = fill_container_greedy(remaining_items, specs['FCL40']['weight'], specs['FCL40']['volume'])
            allocation['FCL40'].append(container_items)
        else:
            # 剩下的货物用LCL
            allocation['LCL'].append(remaining_items)
            break

    # 计算成本
    total_cost = (len(allocation['FCL40']) * costs['FCL40'] +
                  len(allocation['FCL20']) * costs['FCL20'] +
                  sum(container['Volume(CBM)'].sum() for container in allocation['LCL']) * costs['LCL'])

    return allocation, total_cost

# 规格和成本
specs = {
    'FCL40': {'weight': 30480, 'volume': 67},
    'FCL20': {'weight': 14969, 'volume': 33}
}
costs = {
    'FCL40': 5600,
    'FCL20': 3500,
    'LCL': 778
}

# 调用分配函数
allocation, total_cost = allocate_items(df, specs, costs)

# 输出结果
print("Allocated containers:")
for ctype, clist in allocation.items():
    for c in clist:
        print(f"{ctype}: IDs {c['ID'].tolist()}, Total Weight: {c['Weight(kg)'].sum()}, Total Volume: {c['Volume(CBM)'].sum()}")
print(f"Total cost: {total_cost}")


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)


Allocated containers:
FCL40: IDs [17.0, 25.0, 15.0, 23.0, 16.0, 18.0, 22.0, 24.0, 9.0, 1.0, 11.0, 21.0, 2.0, 10.0], Total Weight: 14353.0, Total Volume: 64.4
FCL40: IDs [7.0, 20.0, 3.0, 12.0, 8.0, 19.0, 5.0, 4.0, 14.0], Total Weight: 17751.0, Total Volume: 63.4
FCL20: IDs [6.0, 13.0], Total Weight: 6252.0, Total Volume: 19.6
Total cost: 14700


C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
C:\Users\37605\AppData\Local\Temp\ipykernel_19052\3845512442.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  container_items = container_items.append(item)
